In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_squared_error

AttributeError: partially initialized module 'pandas' has no attribute 'core' (most likely due to a circular import)

In [8]:
df = pd.read_csv("dataset.csv")
print(df.head())
print(df.shape)

NameError: name 'pd' is not defined

In [ ]:
df = df.sort_values("date")
df = df.dropna()  # Remove any missing values

# Keep all features except 'date' and 'next_day_close' as inputs
features = df.drop(["date", "next_day_close"], axis=1)
target = df["next_day_close"]

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_features = scaler.fit_transform(features)
scaled_target = scaler.fit_transform(target.values.reshape(-1, 1))


NameError: name 'df' is not defined

In [ ]:
def create_sequences(X, y, time_steps=60):
    Xs, ys = [], []
    for i in range(time_steps, len(X)):
        Xs.append(X[i-time_steps:i])
        ys.append(y[i])
    return np.array(Xs), np.array(ys)

time_steps = 60
X, y = create_sequences(scaled_features, scaled_target, time_steps)
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

In [ ]:
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

In [ ]:
model = Sequential([
    LSTM(100, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),
    LSTM(100, return_sequences=False),
    Dropout(0.2),
    Dense(50, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.1,
    shuffle=False
)

In [ ]:
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)
y_test_actual = scaler.inverse_transform(y_test)

plt.figure(figsize=(12,6))
plt.plot(y_test_actual, label='Actual')
plt.plot(predictions, label='Predicted')
plt.legend()
plt.show()

rmse = np.sqrt(mean_squared_error(y_test_actual, predictions))
print("RMSE:", rmse)


In [ ]:
model.save("tesla_lstm_model.h5")

In [ ]:
last_60_days = scaled_features[-60:]
X_future = np.array([last_60_days])
predicted_price = model.predict(X_future)
predicted_price = scaler.inverse_transform(predicted_price)
print("Predicted Next Day Close:", predicted_price[0][0])